In [1]:
# libraries
library(RBGL) #required, part of bioconductr
library(Rgraphviz)
library(rbenchmark)
library(data.table) #required
library(tidyverse) #required

Loading required package: graph
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: 'BiocGenerics'

The following objects are masked from 'package:parallel':

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from 'package:stats':

    IQR, mad, sd, var, xtabs

The following objects are masked from 'package:base':

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min

Loading required package: grid
Warning message:
"package 'data.table' was built un

In [2]:
FIPEX_table=read.csv("FIPEX_Advanced_DD_2020.csv")
FIPEX_table

NodeType,NodeEID,NodeLabel,HabQuantity,HabUnits,BarrierPerm,NaturalTF,DownstreamEID,DownstreamNodeLabel,DownstreamNeighDistance,DistanceUnits
<fct>,<int>,<fct>,<dbl>,<fct>,<dbl>,<lgl>,<fct>,<fct>,<dbl>,<fct>
Branch Junction,2,1,44.47,Metres,1.0,FALSE,Sink,Sink,44.47,Metres
Barrier,3,H,3.63,Metres,0.5,FALSE,2,1,3.63,Metres
Barrier,6,I,13.17,Metres,0.0,FALSE,2,1,13.17,Metres
Branch Junction,8,5,18.47,Metres,1.0,FALSE,3,H,18.47,Metres
Source Junction,4,2,8.61,Metres,1.0,FALSE,6,I,8.61,Metres
Barrier,9,G,7.68,Metres,0.5,FALSE,8,5,7.68,Metres
Branch Junction,11,7,24.02,Metres,1.0,FALSE,8,5,24.02,Metres
Barrier,18,M,18.08,Metres,0.0,FALSE,9,G,18.08,Metres
Barrier,13,F,6.55,Metres,0.0,TRUE,11,7,6.55,Metres


In [32]:
################################################################
# edge-weighted connectivity list (edges_all) and matrix - for DCI w/ DD
# nodes / edges are not reversed as in original way
# do not need self-connected

create_advanced_adjmatrix_2020 <- function(FIPEX_table=NULL){

    # get connectivity list, convert columns to characters
    FIPEX_table_DD <- FIPEX_table %>% select(NodeEID,DownstreamEID,DownstreamNeighDistance) %>%
    mutate(temp = as.character(NodeEID)) %>%
    mutate(NodeEID = temp) %>%
    mutate(temp = as.character(DownstreamEID)) %>%
    mutate(DownstreamEID = temp) %>%
    select(-temp) %>%
    mutate(DownstreamEID = ifelse(DownstreamEID == "Sink","sink",DownstreamEID))

    # downstream neighbours
    edges_down <- FIPEX_table_DD

    # upstream neigbours
    edges_up <- FIPEX_table_DD %>% select(NodeEID,DownstreamEID,DownstreamNeighDistance) %>%
    mutate(temp = DownstreamEID) %>%
    mutate(DownstreamEID = NodeEID) %>%
    mutate(NodeEID = temp) %>%
    select(-temp)

    # self-connected
    #FIPEX_table %>% select(NodeEID,DownstreamEID,DownstreamNeighDistance) %>%
    #mutate(NodeEID = DownstreamEID, DownstreamNeighDistance = 0.1)

    # merge
    edges_all <- edges_down %>%
    bind_rows(edges_up) %>%
    #add_row(NodeEID = "sink", DownstreamEID = "sink") %>%
    rename(Node1 = NodeEID, Node2 = DownstreamEID, edgeLength = DownstreamNeighDistance)
    
    # convert to matrix
    adj_matrix_edgelengths <- with(edges_all, tapply(edgeLength, list(Node1, Node2), FUN=sum, default = 0))
    
    return(adj_matrix_edgelengths)
}

In [9]:
# edge weights are used for distance calculations
# edge data / attributes used for habitat quantity calculations
# #https://www.rdocumentation.org/packages/graph/versions/1.50.0/topics/graphAM-class
create_graph_dd_2020 <- function(adj_matrix_edgelengths=0.0,FIPEX_table=NULL){

    # Create graph object
    # 2020 - different way to call the graphAM function 
    # vs pre-2020
    g_dd <- graphAM(adjMat=adj_matrix_edgelengths,  edgemode="directed", values=list(weight=1))

    # associate passabilities with nodes using NodeData slot
    # e.g. nodeData(g,n=c("b", "c"), attr ="color") <- "red"
    nodeDataDefaults(g_dd, attr ="pass") <- 1.0
    nodeData(g_dd,n=as.character(FIPEX_table$NodeEID), attr="pass") <- as.double(FIPEX_table$BarrierPerm)
    #nd <- nodes(g_dd)

    nodeDataDefaults(g_dd, attr ="nodelabel") <- "none"
    nodeData(g_dd,n=as.character(FIPEX_table$NodeEID), attr="nodelabel") <- as.character(FIPEX_table$NodeLabel)
    nodeData(g_dd,n="sink", attr="nodelabel") <- "sink"
    #nd <- nodes(g_dd)

    nodeDataDefaults(g_dd, attr ="downnodelabel") <- "none"
    nodeData(g_dd,n=as.character(FIPEX_table$NodeEID), attr="downnodelabel") <- as.character(FIPEX_table$DownstreamNodeLabel)
    #nd <- nodes(g_dd)

    nodeDataDefaults(g_dd, attr ="natural") <- "none"
    nodeData(g_dd,n=as.character(FIPEX_table$NodeEID), attr="natural") <- FIPEX_table$NaturalTF
    nodeData(g_dd,n="sink", attr="natural") <- FALSE
    #nd <- nodes(g_dd)

    # optionally can give edges attributes
    #edgeDataDefaults(g_dd, attr="name")<-"noname"
    #edgeData(self, from, to, attr)
    #edgeData(self, from, to, attr) <- value
    edgeDataDefaults(g_dd, attr="HabitatQuan")<-0.0
    edgeData(g_dd,from=as.character(FIPEX_table$NodeEID), 
         to=as.character(FIPEX_table$DownstreamEID), 
         attr="HabitatQuan")<-as.double(FIPEX_table$HabQuantity)
    # reverse - attr associated with each direction along one edge
    edgeData(g_dd,from=as.character(FIPEX_table$DownstreamEID), 
         to=as.character(FIPEX_table$NodeEID), 
         attr="HabitatQuan")<-as.double(FIPEX_table$HabQuantity)


    # give edges an easy-to-access name insensitive to direction
    # this is done to quickly identify duplicates later
    # there may be alternatives such as accessing edgeNames but I suspect
    # they are slower than this
    edgeDataDefaults(g_dd, attr="EdgeNameGO")<-"init"
    edgeData(g_dd,from=as.character(FIPEX_table$NodeEID), 
         to=as.character(FIPEX_table$DownstreamEID), 
         attr="EdgeNameGO")<-paste(as.character(FIPEX_table$DownstreamEID),
                                   as.character(FIPEX_table$NodeEID),
                                   sep="-")
    # reverse - attr associated with each direction along one edge
    edgeData(g_dd,from=as.character(FIPEX_table$DownstreamEID), 
         to=as.character(FIPEX_table$NodeEID), 
         attr="EdgeNameGO")<-paste(as.character(FIPEX_table$DownstreamEID),
                                   as.character(FIPEX_table$NodeEID),
                                   sep="-")
    return(g_dd)
}

In [8]:
get_paths_distances <- function(g=NULL,fromnode="sink"){
    dijkstra.sp(g,fromnode,eW=unlist(edgeWeights(g)))
    
    # TO DO: ALTERNATIVES FOR BENCHMARKING
}

In [10]:
get_paths_distances_old <- function(g=NULL,fromnode="sink", tonode=1){
    sp.between(g=g,start=fromnode,finish=tonode, detail=TRUE)
}

In [11]:
FIPEX_table <- FIPEX_table %>%
mutate(DownstreamEID = ifelse(DownstreamEID == "Sink","sink",as.character(DownstreamEID)))
adj_matrix <- create_advanced_adjmatrix_2020(FIPEX_table)
g_dd <- create_graph_dd_2020(adj_matrix,FIPEX_table)



In [13]:
path_benchmark_test <- function(g_dd=NULL,which_algo="djikstra.sp"){
  fromnodecount=numNodes(g_dd)
  for (i in 1:fromnodecount){
    fromnode_name = nodes(g_dd)[i]
    if (which_algo=="djikstra.sp"){
      paths_distances <- get_paths_distances(g_dd,fromnode_name)
    }else{
      for (j in 1:fromnodecount){
        tonode_name = nodes(g_dd)[j]
        paths_distances <- get_paths_distances_old(g_dd,fromnode_name, tonode_name)
      }
    }
  }
}

In [16]:
benchmark(
    paths_distances<-path_benchmark_test(g_dd, "djikstra.sp"),
    paths_distances<-path_benchmark_test(g_dd, "sp.between"),
    replications=10)

test,replications,elapsed,relative,user.self,sys.self,user.child,sys.child
<fct>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
"paths_distances <- path_benchmark_test(g_dd, ""djikstra.sp"")",10,0.58,1.000,0.58,0.00,NA,NA
"paths_distances <- path_benchmark_test(g_dd, ""sp.between"")",10,26.73,46.086,25.72,0.03,NA,NA


NULL

In [46]:
numNodes(g_dd)
fromnode_name = nodes(g_dd)[1]
get_paths_distances(g_dd,fromnode_name)



[1] 36

$distances
    11     12     13     15     16     17     18     19      2     20     23 
  0.00  20.92   6.55  32.20  17.08  23.37  49.78  26.47  46.12  30.40  59.39 
    24     26     27     28      3     30     31     32     33     34     35 
 35.01  36.53  66.91  39.68  42.49  41.98  42.11  47.84  45.94  54.10  63.86 
    36     38      4     40     43     44     45     48     49     50      6 
 57.63  64.02  67.90  69.42  77.55  83.51  80.28  93.62 101.27  93.85  59.29 
     8      9   sink 
 24.02  31.70  90.59 

$penult
  11   12   13   15   16   17   18   19    2   20   23   24   26   27   28    3 
   1    3    1    2    1    5   35    5   16    6    7    8   12   13   10   34 
  30   31   32   33   34   35   36   38    4   40   43   44   45   48   49   50 
  10   13   17   12   20   18   20   21   33   23   24   27   24   28   28   29 
   6    8    9 sink 
   9    1   34    9 

$start
11 
 1